In [2]:
from sknn.mlp import Classifier, Layer, Convolution
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
#from nolearn.dbn import DBN
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from sklearn.svm import LinearSVC
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from PIL import Image
import PIL.ImageOps
from collections import defaultdict
from glob import glob
from random import shuffle, seed
import pylab as pl
import pandas as pd
import re
from sklearn import metrics
from __future__ import print_function
import cv2
import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

ImportError: No module named sknn.mlp

In [2]:
def img_to_matrix(filename):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = cv2.imread(filename,0)
    #img = list(img.getdata())
    #img = map(list, img)
    img = np.array(img)
    #s = img.shape[0] * img.shape[1]
    #img_wide = img.reshape(1, s)
    return img

shirt = glob('D:/python image processing/bag/dataset/shirt_resized/*')
collared_tees = glob('D:/python image processing/bag/dataset/collared_tees_resized/*')
normal_tees = glob('D:/python image processing/bag/dataset/normal_tees_resized/*')

process_file = img_to_matrix

raw_data = [(process_file(filename),'shirt',filename) for filename in shirt] + \
           [(process_file(filename),'collared_tees',filename) for filename in collared_tees] + \
           [(process_file(filename),'normal_tees',filename) for filename in normal_tees]

# randomly order the data
seed(0)
shuffle(raw_data)

In [8]:
data = np.array([cd for (cd,_y,f) in raw_data])
labels = np.array([_y for (cd,_y,f) in raw_data])
y = [(0 if label == 'collared_tees' else( 1 if label=='normal_tees' else 2 )) for label in labels]
#l=[StandardScaler().fit_transform(data) for data in data]
#l=np.asarray(l)
#y=np.asarray(y)
#X_train, X_test, y_train, y_test = train_test_split(l, y, test_size=0.2, random_state=2)

In [20]:

# Create a neural network that uses convolution to scan the input images.
nn = Classifier(
    layers=[
        Convolution('Rectifier', channels=6, kernel_shape=(3, 3), border_mode='full'),
        Convolution('Rectifier', channels=3,pool_type='max',pool_shape=(4,4), kernel_shape=(3, 3), border_mode='valid'),
        Layer("Softmax")],
    learning_rate=0.002,
    valid_size=0.2,
    n_stable=10,
    verbose=True)

nn.fit(X_train, y_train)


Classifier(batch_size=1, debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Convolution `Rectifier`: channels=6, name=u'hidden0', kernel_shape=(3, 3), kernel_stride=(1, 1), pool_shape=(1, 1), border_mode='full'>,
      hidden1=<sknn.nn.Convolution `Rectifier`: channels=3, name=u'hidden1', pool_type='max', kernel_shape=(3, 3), kernel_stride=(4, 4), pool_shape=(4, 4), border_mode='valid'>,
      layers=[<sknn.nn.Convolution `Rectifier`: channels=6, name=u'hidden0', kernel_shape=(3, 3), kernel_stride=(1, 1), pool_shape=(1, 1), border_mode='full'>, <sknn.nn.Convolution `Rectifier`: channels=3, name=u'hidden1', pool_type='max', kernel_shape=(3, 3), kernel_stride=(4, 4), pool_shape=(4, 4), border_mode='valid'>, <sknn.nn.Layer `Softmax`: name=u'output', units=3L>],
      learning_momentum=0.9, learning_rate=0.002, learning_rule=u'sgd',
      loss_type=u'mse', mutator=None, n_iter=None, n_stable=10,
      output=<sknn.nn.Layer `Softmax`: name=u'output', units=3L>,
      ran

In [22]:
preds = nn.predict(X_test)
print (metrics.accuracy_score(y_test, preds))

0.522727272727


In [27]:
from lasagne import layers
from lasagne.nonlinearities import  softmax, rectify
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
import numpy as np

def fit_model(train_x, y, test_x):
    """Feed forward neural network for kaggle digit recognizer competition.
    Intentionally limit network size and optimization time (by choosing max_epochs = 15) to meet runtime restrictions
    """
    print("\n\nRunning Convetional Net.  Optimization progress below\n\n")
    net1 = NeuralNet(
        layers=[  #list the layers here
            ('input', layers.InputLayer),
            ('hidden1', layers.DenseLayer),
            ('hidden2', layers.DenseLayer),
            ('output', layers.DenseLayer),
            ],

        # layer parameters:
        input_shape=(None, train_x.shape[1]),
        hidden1_num_units=200, hidden1_nonlinearity=rectify,  #params of first layer
        hidden2_num_units=200, hidden2_nonlinearity=rectify,
        output_nonlinearity=softmax,  # softmax for classification problems
        output_num_units=3,  # 10 target values

        # optimization method:
        update=nesterov_momentum,
        update_learning_rate=0.05,
        update_momentum=0.7,

        regression=False,
        max_epochs=10,  # Intentionally limited for execution speed
        verbose=1,
        )
    net1.fit(train_x, y)
    predictions = net1.predict(test_x)
    return(predictions)

In [29]:
basic_nn_preds = fit_model(X_train, y_train, X_test)



Running Convetional Net.  Optimization progress below




AttributeError: 'module' object has no attribute 'relu'